In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import obb_login
from openbb import obb
from humbldata.core.utils.env import Env
import polars as pl

# obb_login()
obb.account.login(pat=Env().OBB_PAT, remember_me=True)

In [ ]:

from humbldata.toolbox.technical.mandelbrot_channel.model import (
    calc_mandelbrot_channel,
    calc_mandelbrot_channel_historical,
)


data = (
    obb.equity.price.historical(
        ["AAPL"],
        provider="yfinance",
        start_date="2020-01-01",
        end_date="2024-01-01",
        adjustment="splits_and_dividends",
    ).to_polars()
).drop(["dividends", "stock_splits"]).with_columns(pl.lit("AAPL").alias("symbol"))

mandelbrot = calc_mandelbrot_channel_historical(  # noqa: ERA001
        data,
        window="1m",
        rv_adjustment=True,
        rv_method="std",
        rv_grouped_mean=False,
        rs_method="RS",
        live_price=False,
    ).collect()
mandelbrot

## Test `calc_up_down_pct`


In [ ]:
from humbldata.portfolio.analytics.user_table.helpers import calc_up_down_pct


df = pl.DataFrame({
    "symbol": ["AAPL", "GOOGL", "MSFT"],
    "bottom_price": [5.18, 15.07, 16.24],
    "recent_price": [10.05, 20.31, 16.42],
    "top_price": [11.23, 25.17, 30.09],
})
result = calc_up_down_pct(df)
print(result)

# Setting up UserTable

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.standard_models.portfolio.analytics.user_table import UserTableQueryParams
from humbldata.core.utils.openbb_helpers import get_latest_price


symbols = UserTableQueryParams(symbol="AAPL, MSFT, NVDA").symbol
symbols



get_latest_price(symbols)

# Tesing Async


## Get Sector Async

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import aget_etf_category
import polars as pl

stocks = pl.Series([
    "GOOGL",  # Communication Services
    "AMZN",   # Consumer Discretionary
    "KO",     # Consumer Staples
    "XOM",    # Energy
    "JPM",    # Financials
    "JNJ",    # Health Care
    "HON",    # Industrials
    "AAPL",   # Information Technology
    "LIN",    # Materials
    "PLD",    # Real Estate
    "NEE"     # Utilities
])
test = (await aget_etf_category(symbols=["AAPL", "XLE", "DBA"], provider="yfinance")).collect()
# test2 = (await aget_equity_sector(symbols=stocks, provider="yfinance")).collect()
# test2 = (await aget_equity_sector(symbols=["XLE", "AAPL"], provider="yfinance")).collect()
test

In [ ]:
symbols = ["CORN", 'SLV', 'DBA', "XLE", "AAPL", "FXE", "SPY", "QQQ", "BITO", "MAXI", "SATO", "BIL", "LQD", "QAI", "MNA", "CLSE"]
symbols2 = ["AAPL", "FXE", "SPY", "MAXI", "GLD"]
symbols3 = commodity_etf_symbols = ["GLD", "IAU", "SLV", "GLDM", "PDBC", "SGOL", "FTGC", "DBC", "SIVR", "USO", "IAUM", "GSG", "BCI", "PPLT", "COMT", "GLTR", "OUNZ", "BAR", "UNG", "DBA", "AAAU", "DJP", "CMDT", "KRBN", "PALL", "COM", "NBCM", "CMDY", "DBO", "BCD", "KCCA", "CPER", "HGER", "USCI", "DBB", "DBP", "GCC", "WEAT", "BNO", "UGA", "DGP", "IGLD", "COMB", "DBE", "FGDL", "CORN", "USL", "CCRV", "DJCB", "IAUF", "PLTM", "BDRY", "PDBA", "BGLD", "UCIB", "SOYB", "GRN", "PIT", "BCIM", "RENW", "SDCI", "UNL", "SHNY", "DCMT", "TAGS", "CANE", "TMET", "KEUA", "HCOM", "HARD", "EVMT", "USG", "AMPD", "DZZ", "DGZ", "KMET", "CMCI", "USOY", "ZSC", "TILL", "BWET", "DULL", "ZSB", "LNGG", "USOI", "OILK", "SLVO", "GLDI", "BOIL", "KOLD", "AGQ", "GLL", "UGL", "ZSL", "UCO", "SCO"]



df = (await aget_etf_category(symbols=["AAPL", "XLE", "DBA"])).collect()


In [ ]:
from humbldata.portfolio.analytics.user_table.helpers import normalize_asset_class


normalized_df = normalize_asset_class(df).select(["symbol", "category"])
normalized_df

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import aget_etf_category


df2 = (await aget_etf_category(symbols=["AAPL", "XLE", "DBA"])).collect()
df2

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.standard_models.portfolio.analytics.etf_category import ETFCategoryData


ETFCategoryData(df2)

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.portfolio.analytics.user_table.helpers import aget_asset_class_filter


(await aget_asset_class_filter(symbols=["AAPL", "TSLA", "NVDA"])).collect()

## Test aget_sector_filter

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.portfolio.analytics.user_table.helpers import aget_sector_filter


(await aget_sector_filter(symbols=["XLU", "XLE", "XLF", "XLC", "KO", "AAPL"])).collect()

# Test aggregate_user_table_data

In [ ]:


from humbldata.portfolio.analytics.user_table.helpers import aggregate_user_table_data
import polars as pl

out: pl.LazyFrame = (await aggregate_user_table_data(symbols=["XLU", "XLE", "AAPL"]))
out.collect()

## Test UserTableFetcher

In [6]:
%load_ext autoreload
%autoreload 2

from humbldata.portfolio.portfolio_controller import Portfolio

portfolio = Portfolio(symbols=["AAPL", "TSLA", "NVDA"], user_role="peon")
portfolio

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Portfolio(symbols=['AAPL', 'TSLA', 'NVDA'], provider=yahoo, user_role=peon)

In [2]:
(await portfolio.analytics.user_table()).to_dict(as_series=False)

INFO: UserTableFetcher || START: fetch_data (async)
/Users/jjfantini/github/humblFINANCE-org/humblDATA/menv/lib/python3.11/site-packages/openbb_yfinance/models/etf_info.py:286: UserWarning: AAPL is not an ETF.
  _warn(f"{symbol} is not an ETF.")
/Users/jjfantini/github/humblFINANCE-org/humblDATA/menv/lib/python3.11/site-packages/openbb_yfinance/models/etf_info.py:286: UserWarning: TSLA is not an ETF.
  _warn(f"{symbol} is not an ETF.")
/Users/jjfantini/github/humblFINANCE-org/humblDATA/menv/lib/python3.11/site-packages/openbb_yfinance/models/etf_info.py:286: UserWarning: NVDA is not an ETF.
  _warn(f"{symbol} is not an ETF.")
INFO: MandelbrotChannelFetcher || START: fetch_data (sync)
INFO: MandelbrotChannelFetcher || END: fetch_data (sync) - Total time: 0.3325s
INFO: UserTableFetcher || END: fetch_data (async) - Total time: 1.8907s


{'date': [datetime.date(2024, 7, 9),
  datetime.date(2024, 7, 9),
  datetime.date(2024, 7, 9)],
 'symbol': ['AAPL', 'NVDA', 'TSLA'],
 'buy_price': [222.0119, 126.0257, 234.0127],
 'last_price': [228.09719848632812, 132.44000244140625, 255.625],
 'sell_price': [237.8304, 140.1558, 258.4716],
 'ud_pct': ['-2.67 / +4.27', '-4.84 / +5.83', '-8.45 / +1.11'],
 'ud_ratio': [0.63, 0.83, 7.59],
 'sector': ['Technology', 'Technology', 'Consumer Cyclical'],
 'asset_class': ['Equity', 'Equity', 'Equity']}

In [8]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import aget_latest_price


(await aget_latest_price(symbols=["AAPL", "TSLA", "NVDA"])).collect()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


last_price,symbol
f64,str
228.325,"""AAPL"""
254.995,"""TSLA"""
132.6704,"""NVDA"""
